In [25]:
# 输入数据的参数
_INPUT1='{"name":"input1","type":0,"uri":"tmp_ea2f608964aa41c59b340462308fec2c"}'
_INPUT2='{"name":"input2","type":0,"uri":"tmp_cd8aab065ed34729b31cffa65790616c"}'
_INPUT3='{"name":"input3","type":0,"uri":"tmp_770229d182cc4158a9e3cb65e0ba255b"}'
_INPUT4='{"name":"input4","type":0,"uri":"tmp_4011092c56f24d4aac41d2869c26f0fd"}'

# 输出数据的参数
_OUTPUT='[{"name":"output1","type":25,"uri":"tmp_b7de8487d00743beb4a40c24e924f719"},{"name":"output2","type":0,"uri":"tmp_2629f8357a204691bd88ca6bef528071"},{"name":"output3","type":0,"uri":"tmp_a9c94ec2d984403fa39f1a403a95c88c"},{"name":"output4","type":0,"uri":"tmp_b4f203fbc641427a84e396907493e9ea"}]'

# 自定义参数


In [28]:
# 引入使用库
import pandas as pd
from ustciscrLab_A import get_score
from ustciscrBDL_B import get_score as get_scoreB
import wfio
import random
import numpy as np

In [29]:
#读取训练集数据 所有带标签数据
def get_train_data():
    train_data = wfio.read_dataframe(_INPUT1)
    #删除第一行中文
#     train_data = train_data.drop(0,axis=0,inplace=False)
    return train_data

#读取训练集数据 所有带标签数据
def get_train_data2():
    train_data = wfio.read_dataframe(_INPUT3)
    #删除第一行中文
#     train_data = train_data.drop(0,axis=0,inplace=False)
    return train_data

# train_data = get_train_data()
# print(train_data[:2])

In [4]:
print(get_train_data().describe())

       2017q1fphdsl 2017q1jxje 2017q1jxsl 2017q1kpje 2017q1kps 2017q1kpse  \
count         10000      10000      10000      10000     10000      10000   
unique          188       2477          5       2477        98       2477   
top               0          0          0          0         0          0   
freq           7524       7524       7524       7524      7524       7524   

       2017q1kpsl 2017q1rkse 2017q2fphdsl 2017q2jxje  ...         nsrmc  \
count       10000      10000        10000      10000  ...         10000   
unique          5       1682          187       2502  ...          7563   
top             0          0            0          0  ...  芜湖市商软冠联贸有限公司   
freq         7524       8317         7499       7499  ...            17   

       scjydz      xy   xydl    xyml   xyzl     yc   zcdz   zczby  \
count   10000   10000  10000   10000  10000  10000  10000   10000   
unique   8312     506     77      17    274      2   8350     323   
top        芜湖  其他专业咨询    批发业  批

In [30]:
#读取测试集数据
def get_test_data():
    test_data = wfio.read_dataframe(_INPUT2)
    #删除第一行中文
#     test_data = test_data.drop(0,axis=0,inplace=False)
    return test_data

#读取测试集数据 B榜
def get_test_data2():
    test_data = wfio.read_dataframe(_INPUT4)
    #删除第一行中文
#     test_data = test_data.drop(0,axis=0,inplace=False)
    return test_data
# test_data = get_test_data()
# print(test_data[:2])

In [31]:
# 将标签数据和主数据对应行联系起来
def get_label_data(train_data, label):
    label_data = train_data[train_data['zjnsrsbh'].isin(label['zjnsrsbh'])]
    return label_data

train_data1_original = get_train_data()
train_data2_original = get_train_data2().rename(columns = 
                                                {'bsrxmmp':'cwrysjh','bsrxm':'cwryxm','djrq':'djkyrq','fdbrxmp':'frsjh',
                                                 'xzjd':'jdxz','zczb':'zczby','hy':'xy','hydl':'xydl','hyml':'xyml','hyzl':'xyzl'})
test_data_original = get_test_data().rename(columns = 
                                                {'bsrxmmp':'cwrysjh','bsrxm':'cwryxm','djrq':'djkyrq','fdbrxmp':'frsjh',
                                                 'xzjd':'jdxz','zczb':'zczby','hy':'xy','hydl':'xydl','hyml':'xyml','hyzl':'xyzl'})
test_data2_original = get_test_data2().rename(columns = 
                                                {'bsrxmmp':'cwrysjh','bsrxm':'cwryxm','djrq':'djkyrq','fdbrxmp':'frsjh',
                                                 'xzjd':'jdxz','zczb':'zczby','hy':'xy','hydl':'xydl','hyml':'xyml','hyzl':'xyzl'})

# train_data_all_original = pd.concat([train_data_original, train_data2_original], sort=True)
# print(train_data_all_original.describe())

In [81]:
print(train_data2_original.describe())

       2017q1fphdsl 2017q1jxje 2017q1jxsl 2017q1kpje 2017q1kps 2017q1kpse  \
count         10000      10000      10000      10000     10000      10000   
unique          192       2518          5       2518        98       2518   
top               0        0.0        0.0        0.0         0        0.0   
freq           7483       7483       7483       7483      7483       7483   

       2017q1kpsl 2017q1rkse 2017q2fphdsl 2017q2jxje  ...    xyml   xyzl  \
count       10000      10000        10000      10000  ...   10000  10000   
unique          5       1649          189       2578  ...      19    276   
top           0.0        0.0            0        0.0  ...  批发和零售业   群众团体   
freq         7483       8352         7423       7423  ...    2757   1709   

         jyfw          nsrmc scjydz            jdxz     yc   zcdz  zczby  \
count   10000          10000  10000           10000  10000  10000  10000   
unique   6185           9207   6133              46      2   5972    341   
top  

In [32]:
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing

def normalize(df, key):
    #规格化
#     s = (df[key] - df[key].min())/(df[key].max() - df[key].min())
    s = df[key]
    return s

#  数据转换 只取8列*4季度*3年 
def translate_data(old_data1):
    old_data = old_data1.copy()
    new_data = pd.DataFrame()
    key = 'zjnsrsbh'
    new_data[key] = old_data[key]
    key = 'fddbrxm'
    new_data[key] = old_data[key]
    key = 'cwryxm'
    new_data[key] = old_data[key]
    key = 'nsrmc'
    new_data[key] = old_data[key]
    
    
    key = 'CYRS'.lower()
    old_data[key] = old_data[key].replace('','0').astype('float')
#     old_data[key] = preprocessing.scale(old_data[key]) # 标准化
    old_data[key] = normalize(old_data, key) # 归一化
    new_data[key] = old_data[key]
    
    key = 'zczby'.lower()
    old_data[key] = old_data[key].replace('','0').astype('float')
#     print(old_data[key][0:100])
#     old_data[key] = preprocessing.scale(old_data[key])
#     old_data[key] = normalize(old_data, key)
#     new_data[key] = old_data[key]
#     mm = MinMaxScaler()
#     train_data = mm.fit_transform(train_data) # 归一化
    hz = ['FPHDSL', 'JXJE', 'JXSL', 'KPJE', 'KPS', 'KPSE', 'KPSL', 'RKSE']
    for i in range(3):
        for j in range(4):
            for h in hz:
                key = '201'+str(7+i)+'q'+str(j+1)+h.lower()
                old_data[key] = old_data[key].replace('','0').replace('""""""""""""""', '0')
                old_data[key] = old_data[key].astype('float')
#                 old_data[key] = preprocessing.scale(old_data[key])
                old_data[key] = normalize(old_data, key)
                new_data[key] = old_data[key]
#     print(new_data.describe())
#     new_data = preprocessing.scale(new_data)
#     print(new_data.std(axis=0))
    return new_data
# new_data = translate_data(train_data)

In [33]:

# 检查生僻字模块
import re
def is_rare_name(string):
    pattern = re.compile(u"[~!@#$%^&* ]")
    match = pattern.search(string)
    if match:
        return True
    try:
        string.encode("gb2312")
    except UnicodeEncodeError:
        return True
    return False

# 提取季度数据累加和
def total_quartdata(trainset,quartlist):
    year = 2017
    while year < 2020:
        for i in range(4):
            for key in quartlist:
                #初始值
                if year == 2017 and i == 0:
                    trainset[key] = trainset[str(year)+'q'+str(i+1)+key]
                else:
                    trainset[key] += trainset[str(year)+'q'+str(i+1)+key]
        year = year+1


# 根据先验知识 提取额外的特征
def get_features(trainset):
    # 定义需要累加的列名列表
    quartlist = ['fphdsl','jxje','kpje','kps','kpse','rkse']
    total_quartdata(trainset, quartlist)
    for key in quartlist:
        trainset[key] = normalize(trainset, key)

    # 计算开票数/(核定数+1)
    trainset['kpbhd'] = trainset['kps']/(trainset['fphdsl']+1)
    trainset['kpbhd'] = normalize(trainset, 'kpbhd')

    # 平均每张票金额
    trainset['fppjje'] = trainset['kpje']/(trainset['kps']+1)
    trainset['fppjje'] = normalize(trainset, 'fppjje')

    # 开票金额与开票数标准差
    trainset['kpjebzc'] = 0
    trainset['kpslbzc'] = 0
    year = 2017
    trainset['pjkpje'] = trainset['kpje']/12
    trainset['pjkpsl'] = trainset['kps']/12
    trainset['pjkpje'] = normalize(trainset, 'pjkpje')
    trainset['pjkpsl'] = normalize(trainset, 'pjkpsl')
    while year < 2020:
        for i in range(4):
            trainset['kpjebzc'] += (trainset[str(year)+'q'+str(i+1)+'kpje']-trainset['pjkpje'])*(trainset[str(year)+'q'+str(i+1)+'kpje']-trainset['pjkpje'])
            trainset['kpslbzc'] += (trainset[str(year)+'q'+str(i+1)+'kps']-trainset['pjkpsl'])*(trainset[str(year)+'q'+str(i+1)+'kps']-trainset['pjkpsl'])
        year = year+1
    trainset['kpjebzc'] = np.sqrt(trainset['kpjebzc']/12)
    trainset['kpslbzc'] = np.sqrt(trainset['kpslbzc']/12)
    trainset['kpjebzc'] = normalize(trainset, 'kpjebzc')
    trainset['kpslbzc'] = normalize(trainset, 'kpslbzc')
    
    # 人名相同
    # 构造字典
    namelist = trainset['fddbrxm'].tolist()+trainset['cwryxm'].tolist()
    namelist.sort()
    i = len(namelist)-1
    namedic = {}
    namedic[namelist[i]] = 1
    flag = namelist[i]
    while i >= 0:
        i = i-1
        if namelist[i] == flag:
            namedic[namelist[i]] += 1
        else:
            namedic[namelist[i]] = 1
            flag = namelist[i]

    # 法人或者财务人员与其他公司重合
    frorcwchlist = []
    fddbrxmlist = trainset['fddbrxm'].tolist()
    cwryxmlist = trainset['cwryxm'].tolist()
    for i in range(len(trainset)):
        temp = 0
        if fddbrxmlist[i] == '' or cwryxmlist[i] == '':
            pass
        else:
            temp += namedic[fddbrxmlist[i]]
            temp += namedic[cwryxmlist[i]]
        frorcwchlist.append(temp)
    trainset['frorcwch'] = frorcwchlist
    trainset['frorcwch'] = normalize(trainset, 'frorcwch')

    # 企业名称生僻字
    qymcspzlist = []
    qymclist = trainset['nsrmc'].tolist()
    for i in range(len(trainset)):
        temp = 0
        if is_rare_name(qymclist[i]):
            temp = 1
        qymcspzlist.append(temp)
    trainset['qymcspz'] = qymcspzlist
    trainset['qymcspz'] = normalize(trainset, 'qymcspz')
    
    return trainset



In [ ]:
# 处理训练集不均衡 扩充正例
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

droplist = ['zjnsrsbh', 'fddbrxm', 'cwryxm', 'nsrmc']

x_train1 = translate_data(train_data1_original)
# x_train1 = get_features(x_train1)
for key in droplist:
    x_train1 = x_train1.drop([key], axis=1, inplace=False) # 删除非数值

x_train2 = translate_data(train_data2_original)
# x_train2 = get_features(x_train2)
for key in droplist:
    x_train2 = x_train2.drop([key], axis=1, inplace=False) # 删除非数值

x_train = x_train1
x_train = pd.concat([x_train1, x_train2])  # 连接两个测试集的特征数据
print(x_train.describe())

y_train1 = train_data1_original['yc']
y_train2 = train_data2_original['yc']

y_train = y_train1
y_train = pd.concat([train_data1_original['yc'], train_data2_original['yc']])  # 连接两个测试集的标签数据

test_data = test_data_original
test_data2 = test_data2_original

oversampler=SMOTE(random_state=42, k_neighbors=5)
# X, Y = x_train[:], y_train[:]
X, Y=oversampler.fit_sample(x_train[:], y_train[:])
print('X:{}, Y:{}'.format(len(X), len(Y)))
print('y_train=1:{}'.format(len(y_train[:][y_train[:]=='1'])))
print('Y=1:{}'.format(len(Y[:][Y[:]=='1'])))
x_test = X
y_test = Y

# 采样 训练集和测试集 打乱数据 
# x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
# X = x_train
# Y = y_train
# print('x_train:{} x_test:{}'.format(len(x_train), len(x_test)))
# print('y_train=1:{}'.format(len(y_train[:][y_train[:]=='1'])))

In [76]:
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn import svm
from sklearn import tree
from sklearn import metrics
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import scipy

# 使用MLP训练 分类
def mlp_train():
    _x_train = X
    _y_train = Y.astype('float')
    _x_test = x_test
    _y_test = y_test.astype('float')
#     print(len(_x_train))
    clf = MLPClassifier(activation='identity', solver='adam', hidden_layer_sizes=(9, ),
                    learning_rate='constant', learning_rate_init=0.01, alpha=1e-4,
                    tol=1e-3, max_iter=500, random_state=43, early_stopping=False, shuffle=True)
    clf.fit(_x_train[:], _y_train[:])
    print(type(_y_train[0]))
    prediction = clf.predict(_x_test[:])
    print('The accuracy of the MLP is: {0}, iter times: {1}'.
          format(metrics.accuracy_score(prediction, _y_test[:]), clf.n_iter_))
#     print('w: {}'.format(sorted(abs(clf.coefs_[0]), reverse=True)))
#     print(clf.coefs_[0])
#     print(clf.intercepts_)
#     num_train = 18000
#     print(len(y_train[:num_train][y_train[:num_train] == '0']), len(y_train[:num_train][y_train[:num_train] == '1']))
    return clf

# MLP参数网格搜索
def mlp_search_parameters():
    _x_train = X
    _y_train = Y
    _x_test = x_test
    _y_test = y_test
    mlp = MLPClassifier(activation='relu', solver='adam', hidden_layer_sizes=(9, ),
                    learning_rate='constant', learning_rate_init=0.01, alpha=1e-3,
                    tol=1e-3, max_iter=500, random_state=43, early_stopping=False, shuffle=True)
    
    parameters = {'learning_rate_init':np.linspace(0.0001, 0.1, 10), 'hidden_layer_sizes': ((8, ), (9, ), (10, ))} #超参数选择空间
    clf = RandomizedSearchCV(mlp, parameters, scoring='accuracy', cv=5, n_jobs=-1)
    
    clf.fit(_x_train[:], _y_train[:])
    print(clf.best_estimator_)#最好的模型
    print(clf.best_score_)#最好模型的得分
    print(clf.best_params_)#最好模型的参数
    
    prediction = clf.predict(_x_test[:])
    print('The accuracy of the MLP is: {0}'.
          format(metrics.accuracy_score(prediction, _y_test[:])))
    return clf

# 使用SVM训练
def svm_train():
    _x_train = X
    _y_train = Y
    clf = svm.SVC(C=1.0, kernel='linear', tol=1e-4, gamma='auto', probability=True)
    clf.fit(_x_train[:], _y_train[:])
    
    prediction = clf.predict(_x_train[:])
    print('The accuracy of the SVM is: {0}'.
          format(metrics.accuracy_score(prediction, _y_train[:])))
    return clf

# 决策树训练
def tree_train():
    _x_train = X
    _y_train = Y
    _x_test = x_test
    _y_test = y_test
    clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=25, min_samples_split=10,
                                      min_samples_leaf=1, max_features=43, random_state=42)
    clf = clf.fit(_x_train, _y_train)
    prediction = clf.predict(_x_test[:])
    print('The accuracy of the tree is: {0}'.
          format(metrics.accuracy_score(prediction, _y_test[:])))
    print('feature_importances_:\n{}'.format(str(_x_train.columns.tolist()) + ':' + str(clf.feature_importances_[:])))
    return clf

# 决策树预测
def tree_predict(clf):
    prob = {
        "zjnsrsbh": test_data['zjnsrsbh'],
    }
    verify_data = pd.DataFrame(prob)
    
    _x_test = translate_data(test_data).drop(['zjnsrsbh'], axis=1, inplace=False).astype('float')
#     _x_test = get_features(_x_test)
    prediction = clf.predict_proba(_x_test) # 输出概率
    print(pd.DataFrame(prediction).describe())
    prediction = prediction[:,1]
    
#     print(len(prediction[prediction[:] == 1.0]))  # 输出标签
    print(len(prediction[prediction[:] == 1.0])+len(prediction[prediction[:] == 0.0]))  # 输出标签
#     print(prediction[0:100])
    verify_data['Probability'] = prediction[:]   # 输出标签
    return verify_data

    
def mlp_predict(clf):
    prob = {
        "zjnsrsbh": test_data['zjnsrsbh'],
    }
    verify_data = pd.DataFrame(prob)
    
    _x_test = translate_data(test_data)
#     _x_test = get_features(_x_test)
    for key in droplist:
        _x_test = _x_test.drop([key], axis=1, inplace=False) # 删除非数值
    _x_test = _x_test.astype('float')
#     prediction = clf.predict(_x_test) # 输出标签
    prediction = clf.predict_proba(_x_test)  # 输出概率
    print(pd.DataFrame(prediction).describe())
    prediction = prediction[:,1]
#     prediction[prediction<0.2] = 0
#     prediction[prediction>0.7] = 1
    
    print(len(prediction[prediction[:] < 0.1]))  # 输出标签
    print(len(prediction[prediction[:] > 0.9]))  # 输出标签
    print(len(prediction[prediction[:] == 1.0])+len(prediction[prediction[:] == 0.0]))  # 输出标签
#     print(len(prediction[prediction[:,1] == 1.0])+len(prediction[prediction[:,1] == 0.0]))  # 输出概率
#     print(len(prediction), len(verify_data))
    print(prediction[0:100])
    verify_data['Probability'] = prediction[:]   # 输出标签
#     verify_data['Probability'] = prediction[:,1]  # 输出概率
    return verify_data

# B榜预测
def mlp_predict_B(clf):
    prob = {
        "zjnsrsbh": test_data2['zjnsrsbh'],
    }
    verify_data = pd.DataFrame(prob)
    
    _x_test = translate_data(test_data2)
#     _x_test = get_features(_x_test)
    for key in droplist:  
        _x_test = _x_test.drop([key], axis=1, inplace=False) # 删除非数值
    _x_test = _x_test.astype('float')
#     prediction = clf.predict(_x_test) # 输出标签
    prediction = clf.predict_proba(_x_test)  # 输出概率
    print(pd.DataFrame(prediction).describe())
    prediction = prediction[:,1]
#     prediction[prediction<0.2] = 0
#     prediction[prediction>0.7] = 1
    
    print(len(prediction[prediction[:] < 0.1]))  # 输出标签
    print(len(prediction[prediction[:] > 0.9]))  # 输出标签
    print(len(prediction[prediction[:] == 1.0])+len(prediction[prediction[:] == 0.0]))  # 输出标签
#     print(len(prediction[prediction[:,1] == 1.0])+len(prediction[prediction[:,1] == 0.0]))  # 输出概率
#     print(len(prediction), len(verify_data))
    print(prediction[0:100])
    verify_data['Probability'] = prediction[:]   # 输出标签
#     verify_data['Probability'] = prediction[:,1]  # 输出概率
    return verify_data
    
    
# clf = mlp_search_parameters() # MLP参数网格搜索
clf = mlp_train()  # MLP训练
# clf = svm_train()  #  SVM训练
# clf = tree_train()  # 决策树训练
get_score.post_user_id('10')
get_scoreB.post_user_id('10')

# 提交验证集进行打榜
verify_data = mlp_predict(clf)  # mlp预测
verify_dataB = mlp_predict_B(clf)  # mlp预测
# print(verify_data.describe())
# print(verify_data[:100])
get_score.post_verify_data(verify_data)
get_scoreB.post_verify_data(verify_dataB)

<class 'numpy.float64'>
The accuracy of the MLP is: 0.9823333333333333, iter times: 13
                 0            1
count  6000.000000  6000.000000
mean      0.497932     0.502068
std       0.483990     0.483990
min       0.000000     0.000000
25%       0.000000     0.000000
50%       0.664165     0.335835
75%       1.000000     1.000000
max       1.000000     1.000000
2699
2871
5569
[0.         1.         0.         1.         1.         0.
 1.         0.         1.         1.         0.         1.
 0.         1.         1.         1.         1.         1.
 1.         0.         0.         1.         1.         1.
 0.         0.         1.         0.         1.         0.33117455
 1.         1.         0.         1.         0.         1.
 0.         1.         1.         1.         0.         1.
 0.         0.         1.         0.         1.         1.
 0.         0.33583461 0.         0.         0.         0.
 0.         0.16591471 1.         1.         1.         0.
 1.         

In [ ]:
#用测试集（训练集其中的1000条数据）来实现打分，传输自己的结果和测试集数据
#提交选手id，这一步必须先于模型训练之前，否则判断坐标
get_score.post_user_id('10')
'''
提交随机产生的答案和测试集数据，都是dataframe格式，两者有相同列'zjnsrsbh'，用于拼接
其中user_test_data只有两个字段'zjnsrsbh'和'Probability','Probability'中应为自己模型训练出的结果概率，
概率值应为[0,1]之间
返回结果：选手id，程序开始运行时间，程序结束运行时间，程序耗时，代码分，最终分（代码分和时间分）
测试集分数不参与打榜
'''
_x_train1 = translate_data(train_data1_original)
_x_train2 = translate_data(train_data2_original)
_x_train = pd.concat([_x_train1, _x_train2])  # 连接两个测试集的特征数据
prediction = clf.predict_proba(_x_train[:].drop(['zjnsrsbh'], axis=1, inplace=False))
prediction = prediction[:, 1]
prediction[prediction>=0.5] = 0.99
prediction[prediction<0.5] = 0.01
prob = {
        "zjnsrsbh": _x_train['zjnsrsbh'],
    }
verify_data = pd.DataFrame(prob)
verify_data['Probability'] = prediction

col_n = ['yc', 'zjnsrsbh']
a = pd.DataFrame(train_data1_original,columns = col_n)
b = pd.DataFrame(train_data2_original,columns = col_n)
_y = pd.concat([a, b])

get_score.post_test_data(verify_data, _y)

from sklearn.metrics import log_loss
print('交叉熵损失：%.8f' % (100*(1-log_loss(_y['yc'], prediction))))  # 交叉熵损失 
c = [1, 1, 1, 1, 0]
d = [1, 1, 1, 0.001, 0]
print(log_loss(c, d))

In [202]:
clf = mlp_train()  #训练

#提交选手id，这一步必须先于模型训练之前
#用训练集所有的数据（9999）条来打分，只传输自己得出的结果
#其中user_verify_data只有两个字段'zjnsrsbh'和'Probability','Probability'中应为自己模型训练出的结果概率，
#概率值应为[0,1]之间
get_score.post_user_id('10')

# 提交验证集进行打榜
verify_data = mlp_predict(clf)  #预测
get_score.post_verify_data(verify_data)

The accuracy of the MLP is: 0.997, iter times: 21
                 0             1
count  6000.000000  6.000000e+03
mean      0.670840  3.291600e-01
std       0.430898  4.308985e-01
min       0.000000  2.677151e-18
25%       0.050677  2.172468e-05
50%       0.964497  3.550285e-02
75%       0.999978  9.493234e-01
max       1.000000  1.000000e+00
3249
1574
16
[5.31676207e-03 9.99729606e-01 4.07170459e-06 9.97427870e-01
 1.00000000e+00 7.46051043e-04 2.64607274e-01 1.08484692e-02
 1.00000000e+00 2.36249355e-02 3.76423588e-07 9.99976277e-01
 1.53232806e-11 9.99998988e-01 8.03728187e-01 1.15098803e-02
 9.99999693e-01 4.97816014e-02 1.00000000e+00 1.06163143e-07
 3.61594817e-03 9.99981063e-01 2.01224978e-01 3.98486485e-01
 5.90565113e-07 2.03664920e-02 9.99999771e-01 1.14680920e-03
 9.99994282e-01 1.07062944e-01 1.00000000e+00 8.82009778e-01
 1.76004519e-08 9.98847320e-01 6.93786624e-06 9.99787606e-01
 1.13120033e-05 9.99998054e-01 8.45993497e-01 7.41008460e-02
 2.18841467e-03 3.74673840e-03

In [ ]:
%whos

In [ ]:
!pip list

In [10]:
!pip install imbalanced-learn

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     |████████████████████████████████| 7.0MB 3.6MB/s 
  Found existing installation: scikit-learn 0.21.3
    Uninstalling scikit-learn-0.21.3:
      Successfully uninstalled scikit-learn-0.21.3


In [9]:
!pip install scikit-learn==0.21.3

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     |████████████████████████████████| 6.7MB 3.6MB/s 
ERROR: imbalanced-learn 0.6.1 has requirement scikit-learn>=0.22, but you'll have scikit-learn 0.21.3 which is incompatible.
  Found existing installation: scikit-learn 0.22
    Uninstalling scikit-learn-0.22:
      Successfully uninstalled scikit-learn-0.22


In [43]:
!pip3 install torch==1.3.1+cpu torchvision==0.4.2+cpu -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 111.8MB 2.1MB/s 
     |████████████████████████████████| 13.6MB 780kB/s 


In [230]:
import torch
import torch.utils.data as Data
import torchvision
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn

BATCH_SIZE = 50
nh = 16

class RNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.rnn=nn.LSTM(
            input_size=8,
            hidden_size=nh,
            num_layers=1,
            batch_first=True,
            bidirectional=True,
        )
        self.out=nn.Sequential(
            nn.Linear(in_features=nh*2,out_features=1),
            nn.Sigmoid(),
        )

    def forward(self,x):
        # 一下关于shape的注释只针对单项
        # output: [batch_size, time_step, hidden_size]
        # h_n: [num_layers,batch_size, hidden_size] # 虽然LSTM的batch_first为True,但是h_n/c_n的第一维还是num_layers
        # c_n: 同h_n
        output,(h_n,c_n)=self.rnn(x)
#         print(output.size())
        # output_in_last_timestep=output[:,-1,:] # 也是可以的
        output_in_last_timestep=h_n[-1,:,:]
        # print(output_in_last_timestep.equal(output[:,-1,:])) #ture
        x=self.out(output_in_last_timestep)
        return x

In [ ]:
MODEL_NAME = './model/lstm_model_h64.pth'
PT_MODEL_NAME = './model/lstm_model_h64.pt'
DEVICE = 'cpu'
EPOCHS = 5

if __name__ == "__main__":
    # 1. 加载数据
    dataloader = get_train_data_loader(BATCH_SIZE)
    # showSample(dataloader)
    
    test_dataloader = get_train_data_loader(6000)
    testdata_iter=iter(test_dataloader)
    test_x,test_y=testdata_iter.next()
    print(test_x.shape)
    test_x=test_x.view(-1, 12, 8)
    
    # 2. 网络搭建
    try:
        crnn = torch.jit.load(PT_MODEL_NAME, map_location=DEVICE)  # 通过jit读取模型
        print('load {}'.format(PT_MODEL_NAME))
    except Exception as e:
        print(e)
        net=RNN()
        try:
            net.load_state_dict(torch.load(MODEL_NAME, map_location=DEVICE))
            print('load {}'.format(MODEL_NAME))
        except Exception as e:
            print(e)
    # 3. 网络的训练
    optimizer=torch.optim.Adam(net.parameters(),lr=0.001)
    loss_F=torch.nn.CrossEntropyLoss()
    for epoch in range(EPOCHS): # 数据集只迭代一次
        for step, (x, y) in enumerate(dataloader):
            x_ = x.view(-1, 12, 8)
            pred=net(x_)
#             print(pred)
#             loss=loss_F(pred, y) # 计算loss
            pred = pred.view(-1)
            loss = nn.functional.binary_cross_entropy(pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if step%50==49: # 每50步，计算精度
                with torch.no_grad():
                    test_pred=net(test_x)
#                     prob=torch.nn.functional.softmax(test_pred,dim=1)
#                     pred_cls=torch.argmax(prob,dim=1)
#                     print(list(zip(pred_cls.tolist(), test_y.tolist())))
                    test_pred = test_pred.view(-1)
                    acc = 1 - nn.functional.binary_cross_entropy(test_pred, test_y)
#                     acc=(pred_cls==test_y).sum().numpy()/pred_cls.size()[0]
                    print(f"{epoch}-{step}: accuracy:{acc}")
            if step%100==99: # 每100步，提交一次
                with torch.no_grad():
                    get_score.post_user_id('10')
                    # 提交验证集进行打榜
                    verify_data = lstm_predict(test_data, net)
                    get_score.post_verify_data(verify_data)
                    
                    get_scoreB.post_user_id('10')
                    # 提交验证集进行打榜
                    verify_dataB = lstm_predict(test_data2, net)
                    get_scoreB.post_verify_data(verify_dataB)
                    
                    torch.save(net.state_dict(), MODEL_NAME)  # save
#                     model = torch.jit.script(net)  # 保存模型
#                     model.save(PT_MODEL_NAME)
                    print("save model")
                    

In [51]:
import os
os.remove('./mnist/MNIST/raw/train-images-idx3-ubyte.gz')

In [231]:
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable

transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.5, ], std=[0.5, ])
])

class mydataset(Dataset):

    def __init__(self, x, y, transform=None):
        self.x = x
        self.y = y.astype('float') # float型的结果
        self.transform = transform

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        x = self.x.loc[idx][1:].tolist() # 96维特征
        y = self.y[idx]
        x = torch.from_numpy(np.array(x)).float()
        if self.transform is not None:
            x = self.transform(x)
        return x, y
    
def get_train_data_loader(batch_size):
    dataset = mydataset(X, Y, transform=transform)

    return DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=True,
                      pin_memory=False)

# def get_test_data_loader(batch_size):
#     dataset = mydataset(test_data, transform=transform)

#     return DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)

In [ ]:
X.loc[0][1:]

In [72]:
type(Y[0])

str

In [232]:
# B榜预测
def lstm_predict(_test_data, model):
    prob = {
        "zjnsrsbh": _test_data['zjnsrsbh'],
    }
    verify_data = pd.DataFrame(prob)
    
    _x_test = translate_data(_test_data)
#     _x_test = get_features(_x_test)
    for key in droplist:  
        _x_test = _x_test.drop([key], axis=1, inplace=False) # 删除非数值
    _x_test = _x_test.astype('float')
    _x_test = torch.tensor(_x_test.iloc[:,1:].values, dtype=torch.float32)
    _x_test=_x_test.view(-1, 12, 8)
#     prediction = clf.predict(_x_test) # 输出标签
    test_pred = model(_x_test)  # 输出概率
    test_pred = test_pred.view(-1)
#     prob=torch.nn.functional.softmax(test_pred,dim=1)
#     prediction=torch.argmax(prob,dim=1)
    prediction = test_pred.numpy()
    print(pd.DataFrame(prediction).describe())
#     prediction = prediction[:,1]
#     prediction[prediction<0.2] = 0
#     prediction[prediction>0.7] = 1
    
    print(len(prediction[prediction[:] < 0.1]))  # 输出标签
    print(len(prediction[prediction[:] > 0.9]))  # 输出标签
    print(len(prediction[prediction[:] == 1.0])+len(prediction[prediction[:] == 0.0]))  # 输出标签
#     print(len(prediction[prediction[:,1] == 1.0])+len(prediction[prediction[:,1] == 0.0]))  # 输出概率
#     print(len(prediction), len(verify_data))
    print(prediction[0:10])
    verify_data['Probability'] = prediction[:]   # 输出标签
#     verify_data['Probability'] = prediction[:,1]  # 输出概率
    return verify_data

In [270]:
_x_test = translate_data(test_data2)
#     _x_test = get_features(_x_test)
for key in droplist:  
    _x_test = _x_test.drop([key], axis=1, inplace=False) # 删除非数值
_x_test = _x_test.astype('float')

# 判断96维全0 个数
def len_zero(_data):
    l = 0
    iszero = True
    hz = ['FPHDSL', 'JXJE', 'JXSL', 'KPJE', 'KPS', 'KPSE', 'KPSL', 'RKSE']
    for idx in range(len(_data)):
#         for i in range(3):
#             for j in range(4):
#                 for h in hz:
#                     key = '201'+str(7+i)+'q'+str(j+1)+h.lower()
        iszero = True
        if len(_data.loc[idx][:][_data.loc[idx][:] !=0 ]) > 0:
            iszero = False
        if iszero:      
            l += 1
    return l
print(len_zero(_x_test))
# print(_x_test.iloc[0:1, [1,2,3]])
# _x_test = torch.tensor(_x_test.iloc[:,1:].values, dtype=torch.float32)
# _x_test=_x_test.view(-1, 12, 8)
# print(_x_test[0])

146


In [ ]:
os.mkdir('model')

In [212]:
ls

mnist/  model/


In [217]:
cd ..

/home/nbuser


In [213]:
fd = open(MODEL_NAME, mode="w", encoding="utf-8")
fd.close()

In [215]:
cd model

/home/nbuser/model


In [259]:
np.append(np.arange(2, 98, 1), [0, 2, 3])

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,
       53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
       70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86,
       87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97,  0,  2,  3])